In [1]:
import urllib.request
import csv
import Levenshtein
from difflib import SequenceMatcher
import country_converter as coco
import numpy as np


In [2]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [3]:
def open_list_meetings():
    f = open('statements_count.csv')
    return csv.reader(f)

In [4]:
def extract_statement_count():
    dataset =  open_list_meetings()
    lines = []
    for d in dataset:
        if(len(d) != 0):
            line = d[0].split('\t')
            lines.append((line[0],line[2],line[1]))
    return lines

In [5]:
def extract_entities():
    dataset = extract_statement_count()
    entities = set()
    for d in dataset:
        entities.add(d[0])

    return list(entities)

In [6]:
entities_original = extract_entities()
entities_generated = [p.replace('\n','') for p in list(open("entities_clean.txt"))]

In [7]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [8]:
def extract_initial(entities_gen, dict_e):
    entities_original = extract_entities()
    for eg in entities_gen:
        for e in entities_original:
            d = Levenshtein.ratio(eg,e.upper())
            if(d > 0.9):
                dict_e[e].append(eg)
    
    return dict_e


In [9]:
def extract_title(entities_gen, dict_e):
    entities_original = extract_entities()
    for eg in entities_gen:
        for e in entities_original:
            d = Levenshtein.ratio(eg.title(),e.title())
            if(d > 0.9):
                dict_e[e].append(eg)

    return dict_e

In [10]:
def extract_normal(entities_gen, dict_e):
    entities_original = extract_entities()
    for eg in entities_gen:
        for e in entities_original:
            d1 = Levenshtein.ratio(eg.lower(),e.lower())
            if(d1 > 0.9):
                dict_e[e].append(eg)
    return dict_e

In [11]:
def extract_g77china(entities_gen, dict_e):
    entities_original = extract_entities()
    entities = []
    for eg in entities_gen:
        s = 'G77'
        if(eg == 'G-77/CHINA'):
            dict_e[s].append(eg)
        egs = eg[:3]
        d1 = Levenshtein.ratio(s,egs)
        if(d1 > 0.95):
            dict_e[s].append(eg)

    return dict_e

In [12]:
def create_dict(entities_original):
    dict_e = {}
    for e in entities_original:
        dict_e[e] = []
    return dict_e

In [13]:
def remaining_original(dict_entities):
    list_remain_original = []
    for e in dict_entities:
        if(len(dict_entities[e]) == 0):
           list_remain_original.append(e)
           
    return list_remain_original


In [14]:
def matching(entities_generated, remain_original, dict_entities):
    master_list = entities_generated
    match_these = remain_original
    matching_dict = coco.match(match_these, master_list)
    for e in matching_dict:
        if(matching_dict[e] !='not_found'):
            if(type(matching_dict[e]) ==list):
                dict_entities[e] += matching_dict[e]
            else:
                dict_entities[e].append(matching_dict[e])
                
    return dict_entities


In [15]:
def compute_last_chance(dict_entities):
    entities_generated = [p.replace('\n','') for p in list(open("entities_clean.txt"))]

    remain_original = []
    for e in dict_entities:
        if(len(dict_entities[e]) == 0):
            remain_original.append(e)


    for er in remain_original:
        for e in entities_generated:
            d = Levenshtein.ratio(er.upper(),e)
            if(d > 0.8):
                dict_entities[er].append(e)

    return dict_entities

In [16]:
def compute_dictionary():
    dict_entities = create_dict(entities_original)
    dict_entities = extract_initial(entities_generated, dict_entities)
    dict_entities = extract_title(entities_generated, dict_entities)
    dict_entities = extract_normal(entities_generated, dict_entities)
    dict_entities = extract_g77china(entities_generated, dict_entities)
    remaining_orig = remaining_original(dict_entities)
    dict_entities = matching(entities_generated, remaining_orig , dict_entities)

    dictionary = compute_last_chance(dict_entities)

    for d in dictionary:
        dictionary[d] = list(set(dictionary[d]))

    return dictionary

In [17]:
dictionary = compute_dictionary()